In [1]:
import tensorflow as tf
import numpy as np

Exercise: *Build your own CNN from scratch and try to achieve the highest possible accuracy on MNIST.*

In [2]:
mnist = tf.keras.datasets.mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

11490434/11490434 [==============================] - 0s 0us/step


- The author uses a model with 2 convolutional layers, a max pooling layer, then dropout 25%, dense layer, another dropout but with 50%, and the output layer.
- This model reaches about accuracy in the test set.
- This placed the model roughly in the top 20% of the [MNIST Kaggle competition](https://www.kaggle.com/c/digit-recognizer/).
- You should ignore the models with an accuracy greater than 99.79%, which were most likely trained on the test set, as explained by Chris Deotte in [this post](https://www.kaggle.com/c/digit-recognizer/discussion/61480).

In [3]:
# This is the author's model
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = tf.keras.Sequential(
    [
        tf.keras.layers.Conv2D(
            32,
            kernel_size=3,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal",
        ),
        tf.keras.layers.Conv2D(
            64,
            kernel_size=3,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal",
        ),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(128, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"]
)
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10)
model.evaluate(X_test, y_test)

Epoch 1/10
1719/1719 [==============================] - 22s 7ms/step - loss: 0.1871 - accuracy: 0.9425 - val_loss: 0.0512 - val_accuracy: 0.9848
Epoch 2/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.0809 - accuracy: 0.9755 - val_loss: 0.0380 - val_accuracy: 0.9892
Epoch 3/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.0588 - accuracy: 0.9816 - val_loss: 0.0385 - val_accuracy: 0.9890
Epoch 4/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.0485 - accuracy: 0.9849 - val_loss: 0.0408 - val_accuracy: 0.9886
Epoch 5/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.0399 - accuracy: 0.9874 - val_loss: 0.0395 - val_accuracy: 0.9908
Epoch 6/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.0331 - accuracy: 0.9896 - val_loss: 0.0419 - val_accuracy: 0.9898
Epoch 7/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.0313 - accuracy: 0.9896 - val_loss: 0.0289 - val_ac

[0.03281183913350105, 0.9914000034332275]

According to the same post (link above), we can achieve a higher accuracy (99.5 to 99.7% on the test set), you need to add image augmentation, batch norm, use a learning rate schedule such as 1-cycle, and possibly create an ensemble.

In [7]:
# This is my model
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = tf.keras.Sequential(
    [
        tf.keras.layers.Conv2D(
            32,
            kernel_size=3,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal",
        ),
        tf.keras.layers.Conv2D(
            64,
            kernel_size=3,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal",
        ),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(128, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"]
)
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10)
model.evaluate(X_test, y_test)

Epoch 1/10
1719/1719 [==============================] - 15s 7ms/step - loss: 0.1611 - accuracy: 0.9531 - val_loss: 0.0480 - val_accuracy: 0.9892
Epoch 2/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.0757 - accuracy: 0.9773 - val_loss: 0.0405 - val_accuracy: 0.9886
Epoch 3/10
1719/1719 [==============================] - 11s 7ms/step - loss: 0.0597 - accuracy: 0.9820 - val_loss: 0.0340 - val_accuracy: 0.9908
Epoch 4/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.0453 - accuracy: 0.9860 - val_loss: 0.0320 - val_accuracy: 0.9908
Epoch 5/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.0411 - accuracy: 0.9878 - val_loss: 0.0337 - val_accuracy: 0.9902
Epoch 6/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.0374 - accuracy: 0.9884 - val_loss: 0.0355 - val_accuracy: 0.9898
Epoch 7/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.0305 - accuracy: 0.9899 - val_loss: 0.0332 - val_ac

[0.03181219473481178, 0.9915000200271606]

This notebook is trained on Colab, as it take minutes to train on a CPU.